# CVM. Выбор и проверка критерия для метрик конверсии

## Цели и описание ноутбука
Эксперименты CVM, целевыми метриками которых являются метрики конверсии, могут помочь при решении следующих задач:
- Сравнение конверсии группы с константным значением конверсии
- Сравнение больших конверсий между группами
- Сравнение малых конверсий между группами

Целями данного ноутбука являются:
1. Описание критериев, которые необходимо применять в каждой из вышеописанных задач
2. Описание функций, которые рассчитывают размер выборки и минимальный детектируемый эффект для каждого критерия
3. Описание функций, которые проверяют корректность выбора критерия на синтетических данных
4. Описание функций, которые строят доверительные интервалы для разницы между целевыми метриками
5. Опиание функций, которые используют для проверки статистической значимости при множественном тестировании

In [5]:
# Magic function to autoreload kernel when python module's code is changed
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('white')

## 1. Критерии для метрик конверсий

Критерий (или статистический тест) — это правило, по которому на основании выборочных данных принимается решение: принимать или отвергать нулевую гипотезу $H_0$. При этом критерий основывается на случайной величине $T(X_1, X_2, ... , X_n)$, называемой статистикой критерия.

Основные условия, которым должна удовлетворять случайная величина, чтобы считаться критерием:
1. Определена как функция от выборки
2. Известно (или выводимо) распределение при $H_0$
3. Чувствительность к отклонению от $H_0$ - при верной альтернативе $H_1$ распределение $T$ должно изменяться таким образом, чтобы вероятность попасть в критическую область возрастала
4. Монотонность / направленность - для односторонних тестов значение $T$ должно возрастать (или убывать) при переходе от $H_0$ к $H_1$, чтобы можно было задать однозначную «область отклонения» (чем больше отклонение, тем больше мощность теста)
5. Независимость от параметров, не проверяемых гипотезой (инвариантность)
6. Статистика критерия должна быть вычислимой

При решении задач сравнения конверсий необходимо применять следующие критерии:
1. Сравнение конверсии в группе с константным значением:
    - Биномиальный тест (Binomial test)
    - Одновыборочный T-тест (One-sample T-test)
2. Сравнение больших конверсий между группами:
    - Z-тест для пропорций (Z-test)
    - Двухвыборочный T-тест Уэлча (Welch's T-test)
3. Сравнение малых конверсий между группами:
    - Точный тест Фишера (Fisher's Exact Test)
    - Бутстрап (Bootstrap) и Тест перестановок (применим ко всем трём задачам)

### 1.1.1. Сравнение конверсии в группе с константой. **Биномиальный тест**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: p = p_0 \\
H_1: p <> p_0
$$
где $p$ - конверсия в экспериментальной группе, $p_0$ - константа, с которой необходимо сравнить конверсию

#### Условия применимости критерия
1. Результаты экспериментов - независимые испытания Бернулли
2. Не требует большого размера выборки $n$, критерий работает при малых конверсиях $p$

#### Статистика критерия
Статистикой критерия является количество успехов $k$ в $n$ испытаниях. При верности $H_0$ статистика имеет биномиальное распределение с параметрами $n$ и $p_0$. $k ~ Binomial(n, p_0)$. 

#### Расчёт p-value
$p-value$ в данном случае есть сумма вероятностей всех исходов, вероятность которых меньше либо равна вероятности наблюдаемого $k$.

**Тренировочная задача №1**

**Условие:** представим, что историческая конверсия в целевое действие у сегмента равна $p_0 = 0.7$%. Мы отправляем выборке из 1000 клиентов СМС с призывом совершить целевое действие. После эксперимента мы выяснили, что в экспериментальной группе в целевое действие сконвертировалось 14 клиентов.

**Задача:** сравнить конверисю в экспериментальной группе ($p$) с исторической конверсией по сегменту ($p_0$) при помощи **Биномиального теста** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $p$ не равна $p_0$):
$$
H_0: p = 0.007 \\
H_1: p <> 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p$ больше $p_0$):
$$
H_0: p \le 0.007 \\
H_1: p > 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p$ меньше $p_0$):
$$
H_0: p \ge 0.007 \\
H_1: p < 0.007 \\
$$

In [7]:
from ab_utils.binomial_test import binomial_p_value, binomial_p_value_manual

In [8]:
n = 1000          # количество пользователей (испытаний)
k = 14            # число конверсий (успехов)
p0 = 0.007        # ожидаемая конверсия 0.7% = 0.007
alpha = 0.05      # уровень значимости alpha

p = k / n
print(f"Наблюдаемая конверсия: {p:.1%}")
print(f"Ожидаемая конверсия (H0): {p0:.1%}")

Наблюдаемая конверсия: 1.4%
Ожидаемая конверсия (H0): 0.7%


In [9]:
alt_to_h_zero = {
    'two-sided': f'p {round(p * 100.0, 1)}% равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% больше либо равна p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% меньше либо равна p_0 {round(p0 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p {round(p * 100.0, 1)}% НЕ равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% меньше p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% больше p_0 {round(p0 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_scipy = binomial_p_value(k, n, p0, alternative = alt)
    p_manual = binomial_p_value_manual(k, n, p0, alternative = alt)

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (scipy):  {p_scipy:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_scipy < alpha else 'нет'}")
    print(f"p-value (manual): {p_manual:.6f}")


Альтернатива: two-sided
p-value (scipy):  0.019676; Гипотеза H_0 (p 1.4% равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% НЕ равна p_0 0.7%): да
p-value (manual): 0.019676

Альтернатива: less
p-value (scipy):  0.994456; Гипотеза H_0 (p 1.4% больше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% меньше p_0 0.7%): нет
p-value (manual): 0.994456

Альтернатива: greater
p-value (scipy):  0.012514; Гипотеза H_0 (p 1.4% меньше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% больше p_0 0.7%): да
p-value (manual): 0.012514


### 1.1.2. Сравнение конверсии в группе с константой. **Одновыборочный T-тест**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: \mu = \mu_0 \\
H_1: \mu <> \mu_0
$$
где $\mu$ - среднее в экспериментальной группе, $\mu_0$ - константа, с которой необходимо сравнить среднее

#### Условия применимости критерия
1. Наблюдения независимы
2. Выборка из нормального распределения, либо $n$ достаточно велико ($n > 100$), чтобы (согласно ЦПТ) среднее было нормально распределено

#### Статистика критерия
Пусть
- $\bar{X}$ - выборочное среднее (среднее в экспериментальной группе),
- $s^2$ - несмещённая дисперсия,
- $n$ - размер выборки.

Статистикой критерия является:

$$
t = \frac{\bar{X} - \mu_0}{s / \sqrt{n}}
$$

При верности нулевой гипотезы $H_0$ статистика критерия имеет распределение Стьюдента с $n - 1$ степенями свободы:

$$
t \sim T_{n-1}
$$

#### Расчёт p-value
$p-value$ (вероятность получить такое же или более экстремальное значение статистики) равно:

Для двусторонней альтернативы:
$$
p = 2 \times Probability(T_{n-1} \ge |t|)
$$

Для односторонней альтернативы (больше):
$$
p = Probability(T_{n-1} \ge t)
$$

Для односторонней альтернативы (меньше):
$$
p = Probability(T_{n-1} \le t)
$$

**Тренировочная задача №2**

Возьмём условие **тренировочной задачи №1**, но теперь для сравнения используем **Одновыборочный Т-тест**. **Одновыборочный Т-тест** может быть применён так как $n = 1000$ (достаточно велико для применения ЦПТ).

**Условие:** Историческая конверсия в целевое действие у сегмента равна $p_0 = \mu_0 = 0.7$%. Мы отправляем выборке из 1000 клиентов СМС с призывом совершить целевое действие. После эксперимента мы выяснили, что в экспериментальной группе в целевое действие сконвертировалось 14 клиентов.

**Задача:** сравнить конверисю в экспериментальной группе ($p$, которое равно $\mu$) с исторической конверсией по сегменту ($p_0$, которое равно $\mu_0$) при помощи **одновыборочного Т-теста** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $\mu$ не равна $\mu_0$):
$$
H_0: \mu = 0.007 \\
H_1: \mu <> 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $\mu$ больше $\mu_0$):
$$
H_0: \mu \le 0.007 \\
H_1: \mu > 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $\mu$ меньше $\mu_0$):
$$
H_0: \mu \ge 0.007 \\
H_1: \mu < 0.007 \\
$$

In [10]:
from ab_utils.t_test import onesample_p_value, onesample_p_value_manual

In [11]:
n = 1000          # количество пользователей (испытаний)
k = 14            # число конверсий (успехов)
mu0 = 0.007       # ожидаемая конверсия 0.7% = 0.007
alpha = 0.05      # уровень значимости alpha

mu = k / n
print(f"Наблюдаемая конверсия: {mu:.1%}")
print(f"Ожидаемая конверсия (H0): {mu0:.1%}")

# Создаём массив, в котором 14 клиентов из 1000 конвертировалось в целевое действие
x_arr = np.zeros(n)
x_arr[:k] = 1

Наблюдаемая конверсия: 1.4%
Ожидаемая конверсия (H0): 0.7%


In [12]:
alt_to_h_zero = {
    'two-sided': f'p {round(p * 100.0, 1)}% равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% больше либо равна p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% меньше либо равна p_0 {round(p0 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p {round(p * 100.0, 1)}% НЕ равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% меньше p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% больше p_0 {round(p0 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_scipy = onesample_p_value(x_arr, mu0, alternative = alt)[0]
    p_manual = onesample_p_value_manual(x_arr, mu0, alternative = alt)[0]

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (scipy):  {p_scipy:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_scipy < alpha else 'нет'}")
    print(f"p-value (manual): {p_manual:.6f}")


Альтернатива: two-sided
p-value (scipy):  0.059974; Гипотеза H_0 (p 1.4% равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% НЕ равна p_0 0.7%): нет
p-value (manual): 0.059974

Альтернатива: less
p-value (scipy):  0.970013; Гипотеза H_0 (p 1.4% больше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% меньше p_0 0.7%): нет
p-value (manual): 0.970013

Альтернатива: greater
p-value (scipy):  0.029987; Гипотеза H_0 (p 1.4% меньше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% больше p_0 0.7%): да
p-value (manual): 0.029987


**Обратите внимание** на результат **Одновыборочного Т-теста** при тестировании двусторонней гипотезы - результат отличается от результата **Биномиального теста**. При таких маленьких конверсиях, **Одновыборочный Т-тест** не подходит, так как не выполняется требование о нормальном распределении среднего. Как проверять корректность выбранного критерия будет рассказано в секции "3. Проверка корректности выбора критерия"

### 1.2.1. Сравнение больших конверсий между группами. **Z-тест для пропорций (Z-test)**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: p_1 = p_2 \\
H_1: p_1 <> p_2
$$
где $p_1$ - значение конверсии в экспериментальной группе, $p_2$ - значение конверсии в контрольной группе

#### Условия применимости критерия
1. Испытания независимы
2. $n_1 \times \hat{p_1} \ge 50$, $n_1 \times (1 - \hat{p_1}) \ge 50$, $n_2 \times \hat{p_2} \ge 50$, $n_2 \times (1 - \hat{p_1}) \ge 50$. В противном случае лучше применять "Точный тест Фишера"

#### Статистика критерия
Пусть
- $x_1$, $x_2$ - количество успехов,
- $n_1$, $n_2$ - размеры выборок,
- $\hat{p_1} = \frac{x_1}{n_1}$, $\hat{p_2} = \frac{x_2}{n_2}$,
- объединённая пропорция $\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$.

Статистикой критерия является:

$$
z = \frac{\hat{p_1} - \hat{p_2}}{\sqrt{\hat{p} \times (1 - \hat{p}) \times (\frac{1}{n_1} + \frac{1}{n_2})}}
$$

При верности нулевой гипотезы $H_0$ статистика критерия имеет нормальное распределение со средним равным $0$ и стандартным отклонением равным $1$:

$$
z \sim N(0, 1)
$$

#### Расчёт p-value
$p-value$ (вероятность получить такое же или более экстремальное значение статистики) равно:

Для двусторонней альтернативы:
$$
p = 2 \times Probability(N(0, 1) \ge |z|)
$$

Для односторонней альтернативы (больше):
$$
p = Probability(N(0, 1) \ge z)
$$

Для односторонней альтернативы (меньше):
$$
p = Probability(N(0, 1) \le z)
$$

**Тренировочная задача №3**

**Условие:** мы проверяем эффективность Push-уведомления. Целевым действием является подписка на сервис в течение трёх дней. Мы разделили сегмент на две группы, экспериментальную и контрольную, размер каждой - 5 тысяч клиентов. Экспериментальная группа получила Push-уведомление, контрольная - ничего не получала. После проведения эксперимента мы измерили количество клиентов, подписавшихся на сервис: в контрольной группе оно составило 328 клиентов, а в экспериментальной - 385 клиента.

**Задача:** сравнить конверисю в экспериментальной группе ($p_1$) с конверсией в контрольной группе ($p_2$) при помощи **Z-теста для пропорций** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $p_1$ не равна $p_2$):
$$
H_0: p_1 = p_2 \\
H_1: p_1 <> p_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p_1$ больше $p_2$):
$$
H_0: p_1 \le p_2 \\
H_1: p_1 > p_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p_1$ меньше $p_2$):
$$
H_0: p_1 \ge p_2 \\
H_1: p_1 < p_2 \\
$$

In [13]:
from ab_utils.z_test import ztest_prop_p_value, ztest_prop_p_value_manual

In [14]:
n1 = 5000          # количество пользователей (испытаний) в экспериментальной группе
n2 = 5000          # количество пользователей (испытаний) в контрольной группе

x1 = 385          # количество успехов в экспериментальной группе
x2 = 328          # количество успехов в контрольной группе
alpha = 0.05      # уровень значимости alpha

p1 = float(x1) / n1     # Конверсия в экспериментальной группе
p2 = float(x2) / n2     # Конверсия в контрольной группе

print(f"Конверсия в экспериментальной группе: {p1:.3%}")
print(f"Конверсия в контрольной группе (H0): {p2:.3%}")

# Создаём массив, для каждой группы
x1_arr = np.zeros(n1)
x1_arr[:x1] = 1

x2_arr = np.zeros(n2)
x2_arr[:x2] = 1

Конверсия в экспериментальной группе: 7.700%
Конверсия в контрольной группе (H0): 6.560%


In [15]:
alt_to_h_zero = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% больше либо равна p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% меньше либо равна p_2 {round(p2 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% НЕ равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% меньше p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% больше p_2 {round(p2 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_scipy = ztest_prop_p_value(x1_arr = x1_arr, x2_arr = x2_arr, alternative = alt)[0]
    p_manual = ztest_prop_p_value_manual(x1 = x1, n1 = n1, x2 = x2, n2 = n2, alternative = alt)[0]

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (sm):  {p_scipy:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_scipy < alpha else 'нет'}")
    print(f"p-value (manual): {p_manual:.6f}")


Альтернатива: two-sided
p-value (sm):  0.026716; Гипотеза H_0 (p_1 7.7% равна p_2 6.6%) отвергается на уровне 0.05 в пользу H_1 (p_1 7.7% НЕ равна p_2 6.6%): да
p-value (manual): 0.026754

Альтернатива: less
p-value (sm):  0.986642; Гипотеза H_0 (p_1 7.7% больше либо равна p_2 6.6%) отвергается на уровне 0.05 в пользу H_1 (p_1 7.7% меньше p_2 6.6%): нет
p-value (manual): 0.986623

Альтернатива: greater
p-value (sm):  0.013358; Гипотеза H_0 (p_1 7.7% меньше либо равна p_2 6.6%) отвергается на уровне 0.05 в пользу H_1 (p_1 7.7% больше p_2 6.6%): да
p-value (manual): 0.013377


### 1.2.2. Сравнение больших конверсий между группами. **Двухвыборочный T-тест Уэлча (Welch's T-test)**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: \mu_1 = \mu_2 \\
H_1: \mu_1 <> \mu_2
$$
где $\mu_1$ - значение среднего в экспериментальной группе, $\mu_2$ - значение среднего в контрольной группе

#### Условия применимости критерия
1. Наблюдения в группах независимы
2. Внутри каждой группы наблюдения независимы и идентично распределены
3. Распределения примерно нормальные или выборки достаточно большие (ЦПТ сгладит)
4. Допускается разная дисперсия между группами (это и есть фича Welch)

#### Статистика критерия
Пусть
- $\bar{X_1}$, $\bar{X_2}$ - средние,
- $n_1$, $n_2$ - размеры выборок,
- $s_1^2$, $s_2^2$ - несмещённые дисперсии.

Статистикой критерия является:

$$
t = \frac{\bar{X_1} - \bar{X_2}}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$

При верности нулевой гипотезы $H_0$ статистика критерия имеет распределение Стьюдента с количеством степеней свободы, рассчитанным по формуле Уэлча–Саттертуэйта:

$$
t \sim T_{\nu} \\
\\
\nu = \frac{(\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2})^2}{\frac{s_1^4}{n_1^2 \times (n_1 - 1)} + \frac{s_2^4}{n_2^2 \times (n_2 - 1)}}
$$

#### Расчёт p-value
$p-value$ (вероятность получить такое же или более экстремальное значение статистики) равно:

Для двусторонней альтернативы:
$$
p = 2 \times Probability(T_{\nu} \ge |t|)
$$

Для односторонней альтернативы (больше):
$$
p = Probability(T_{\nu} \ge t)
$$

Для односторонней альтернативы (меньше):
$$
p = Probability(T_{\nu} \le t)
$$

**Тренировочная задача №4**

Возьмём условие **тренировочной задачи №3**, но теперь для сравнения используем **Двухвыборочный Т-тест Уэлча**. **Двухвыборочный Т-тест Уэлча** может быть применён так как $n = 5000$ (достаточно велико для применения ЦПТ).

**Условие:** мы проверяем эффективность Push-уведомления. Целевым действием является подписка на сервис в течение трёх дней. Мы разделили сегмент на две группы, экспериментальную и контрольную, размер каждой - 5 тысяч клиентов. Экспериментальная группа получила Push-уведомление, контрольная - ничего не получала. После проведения эксперимента мы измерили количество клиентов, подписавшихся на сервис: в контрольной группе оно составило 328 клиентов, а в экспериментальной - 385 клиента.

**Задача:** сравнить конверисю в экспериментальной группе ($\mu_1 = p_1$) с конверсией в контрольной группе ($\mu_2 = p_2$) при помощи **Двухвыборочного Т-теста Уэлча** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $\mu_1$ не равна $\mu_2$):
$$
H_0: \mu_1 = \mu_2 \\
H_1: \mu_1 <> \mu_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $\mu_1$ больше $\mu_2$):
$$
H_0: \mu_1 \le \mu_2 \\
H_1: \mu_1 > \mu_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $\mu_1$ меньше $\mu_2$):
$$
H_0: \mu_1 \ge \mu_2 \\
H_1: \mu_1 < \mu_2 \\
$$

In [16]:
from ab_utils.t_test import welch_p_value, welch_p_value_manual

In [17]:
n1 = 5000          # количество пользователей (испытаний) в экспериментальной группе
n2 = 5000          # количество пользователей (испытаний) в контрольной группе

x1 = 385          # количество успехов в экспериментальной группе
x2 = 328          # количество успехов в контрольной группе
alpha = 0.05      # уровень значимости alpha

mu1 = float(x1) / n1     # Конверсия в экспериментальной группе
mu2 = float(x2) / n2     # Конверсия в контрольной группе

print(f"Конверсия в экспериментальной группе: {mu1:.3%}")
print(f"Конверсия в контрольной группе (H0): {mu2:.3%}")

# Создаём массив, для каждой группы
x1_arr = np.zeros(n1)
x1_arr[:x1] = 1

x2_arr = np.zeros(n2)
x2_arr[:x2] = 1

Конверсия в экспериментальной группе: 7.700%
Конверсия в контрольной группе (H0): 6.560%


In [18]:
alt_to_h_zero = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% больше либо равна p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% меньше либо равна p_2 {round(p2 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% НЕ равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% меньше p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% больше p_2 {round(p2 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_scipy = welch_p_value_manual(x = x1_arr, y = x2_arr, alternative = alt)[0]
    p_manual = welch_p_value_manual(x = x1_arr, y = x2_arr, alternative = alt)[0]

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (scipy):  {p_scipy:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_scipy < alpha else 'нет'}")
    print(f"p-value (manual): {p_manual:.6f}")


Альтернатива: two-sided
p-value (scipy):  0.026754; Гипотеза H_0 (p_1 7.7% равна p_2 6.6%) отвергается на уровне 0.05 в пользу H_1 (p_1 7.7% НЕ равна p_2 6.6%): да
p-value (manual): 0.026754

Альтернатива: less
p-value (scipy):  0.986623; Гипотеза H_0 (p_1 7.7% больше либо равна p_2 6.6%) отвергается на уровне 0.05 в пользу H_1 (p_1 7.7% меньше p_2 6.6%): нет
p-value (manual): 0.986623

Альтернатива: greater
p-value (scipy):  0.013377; Гипотеза H_0 (p_1 7.7% меньше либо равна p_2 6.6%) отвергается на уровне 0.05 в пользу H_1 (p_1 7.7% больше p_2 6.6%): да
p-value (manual): 0.013377


### 1.3.1. Сравнение малых конверсий между группами. **Точный тест Фишера (Fisher's Exact Test)**
При помощи критерия можно сравнить две пропорции, как при помощи Z-тест, но без нормальных приближений.

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: p_1 = p_2 \\
H_1: p_1 <> p_2
$$
где $p_1$ - значение конверсии в экспериментальной группе, $p_2$ - значение конверсии в контрольной группе.
Конверсии будут вычислены по следующей таблице:
|           | success | fail | total |
|:----------|:--------:|:----:|:-----:|
| **Group A** | $a$ | $n_1 - a$ | $n_1$ |
| **Group B** | $b$ | $n_2 - b$ | $n_2$ |

#### Условия применимости критерия
1. Две независимые бинарные выборки
2. Подходит при очень редких событиях и маленьких счетчиках

#### Статистика критерия
Fisher не использует $Z$ или $t$ приближения. Он рассчитывает точную гипергеометрическую вероятность всех таких таблиц с теми же маргиналиями (фиксированные суммы по строкам и столбцам) и считает $p-value$ как сумму вероятностей таблиц не менее экстремальных, чем наблюдаемая. При фиксированных $n_1$, $n_2$ и общем числе успехов $a + b$, вероятность получить ровно a успехов в группе A имеет гипергеометрическое распределение.

Пусть есть следующая таблица сопряжённости размера 2х2:
$$
\begin{array}{c|c|c|c}
\text{} & \text{Успех} & \text{Неудача} & \text{Всего} \\ \hline
\textbf{Группа 1} & a & b & n_1 = a + b \\
\textbf{Группа 2} & c & d & n_2 = c + d \\ \hline
\textbf{Всего} & m_1 = a + c & m_2 = b + d & N = n_1 + n_2
\end{array}
$$

Статистикой критерия является число успехов $a$ в первой группе. При фиксированных маргинальных суммах $n_1, n_2, m_1, m_2$ число успехов $a$ в первой группе подчиняется гипергеометрическому распределению:

$$
P(A = a) = \frac{\binom{m_1}{a} \times \binom{m_2}{n_1 - a}}{\binom{N}{n_1}}
$$
где
- $\binom{m_1}{a}$ - число способов выбрать $a$ успехов из всех успехов,
- $\binom{m_2}{n_1 - a}$ - число способов выбрать остальные неудачи,
- $\binom{N}{n_1}$ - общее число способов выбрать $n_1$ наблюдений для первой группы.
Статистикой критерия Фишера является вероятность конкретной таблицы при верности $H_0$.

При верности нулевой гипотезы $H_0: p_1 = p_2 = p$ количество успехов $A$ в первой группе имеет гипергеометрическое распределение:

$$
A \sim Hypergeometric(N, m_1, n_1) \\
$$
где
- $N$ - общее число наблюдений,
- $m_1$ - общее число успехов,
- $n_1$ - число наблюдений в первой группе.

#### Расчёт p-value
$p-value$ (вероятность получить такое же или более экстремальное значение статистики) - сумма гипергеометрических вероятностей "не менее экстремальных, чем наблюдаемая таблица":

Для двусторонней альтернативы:
$$
p-value = \sum_{a' \in E} P(A = a')
$$
где $E$ - множество всех $a'$, для которых наблюдаемая ассоциация между группами не менее сильная, чем в данных (в терминах отношения шансов или отклонения от независимости).

**Тренировочная задача №5**

**Условие:** мы проверяем эффективность Push-уведомления. Целевым действием является подписка на сервис в течение трёх дней. Мы разделили сегмент на две группы, экспериментальную и контрольную, размер каждой - 1 тысяча клиентов. Экспериментальная группа получила Push-уведомление, контрольная - ничего не получала. После проведения эксперимента мы измерили количество клиентов, подписавшихся на сервис: в контрольной группе оно составило 8 клиентов, а в экспериментальной - 21 клиент.

**Задача:** сравнить конверисю в экспериментальной группе ($p_1$) с конверсией в контрольной группе ($p_2$) при помощи **Точного теста Фишера** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $p_1$ не равна $p_2$):
$$
H_0: p_1 = p_2 \\
H_1: p_1 <> p_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p_1$ больше $p_2$):
$$
H_0: p_1 \le p_2 \\
H_1: p_1 > p_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p_1$ меньше $p_2$):
$$
H_0: p_1 \ge p_2 \\
H_1: p_1 < p_2 \\
$$

In [19]:
from ab_utils.fishers_exact_test import fishers_exact_p_value

In [20]:
n1 = 1000          # количество пользователей (испытаний) в экспериментальной группе
n2 = 1000          # количество пользователей (испытаний) в контрольной группе

x1 = 21            # количество успехов в экспериментальной группе
x2 = 8             # количество успехов в контрольной группе
alpha = 0.05       # уровень значимости alpha

p1 = float(x1) / n1     # Конверсия в экспериментальной группе
p2 = float(x2) / n2     # Конверсия в контрольной группе

a = x1          # Число успехов в тестовой группе
b = n1 - x1     # число неудач в тестовой группе
c = x2          # число успехов в контрольной группе
d = n2 - x2     # число неудач в контрольной группе

print(f"Конверсия в экспериментальной группе: {p1:.1%}")
print(f"Конверсия в контрольной группе (H0): {p2:.1%}")

Конверсия в экспериментальной группе: 2.1%
Конверсия в контрольной группе (H0): 0.8%


In [21]:
alt_to_h_zero = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% больше либо равна p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% меньше либо равна p_2 {round(p2 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% НЕ равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% меньше p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% больше p_2 {round(p2 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_scipy = fishers_exact_p_value(
        a = a,
        b = b,
        c = c,
        d = d,
        alternative = alt
    )[0]

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (scipy):  {p_scipy:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_scipy < alpha else 'нет'}")


Альтернатива: two-sided
p-value (scipy):  0.023120; Гипотеза H_0 (p_1 2.1% равна p_2 0.8%) отвергается на уровне 0.05 в пользу H_1 (p_1 2.1% НЕ равна p_2 0.8%): да

Альтернатива: less
p-value (scipy):  0.996156; Гипотеза H_0 (p_1 2.1% больше либо равна p_2 0.8%) отвергается на уровне 0.05 в пользу H_1 (p_1 2.1% меньше p_2 0.8%): нет

Альтернатива: greater
p-value (scipy):  0.011560; Гипотеза H_0 (p_1 2.1% меньше либо равна p_2 0.8%) отвергается на уровне 0.05 в пользу H_1 (p_1 2.1% больше p_2 0.8%): да


### 1.3.2. Сравнение малых конверсий между группами. **Бутстрап (Bootstrap)**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: \theta_1 = \theta_2 \\
H_1: \theta_1 <> \theta_2
$$
где $\theta_1$ - значение метрики (любой) в экспериментальной группе, $\theta_2$ - значение метрики (любой) в контрольной группе

#### Условия применимости критерия
1. Независимость наблюдений

#### Статистика критерия
Пусть
- $\hat{\theta_1}$ - значение метрики в экспериментальной группе,
- $\hat{\theta_2}$ - значение метрики в контрольной группе.

Статистикой критерия является:

$$
T_{obs} = \hat{\theta_1} - \hat{\theta_2}
$$

Дальше:
1. Строим бутстрап-распределение разницы (либо непараметрический перестановочный тест / permutation test).
2. Оцениваем, насколько наблюдаемое T экстремально.
Делаем permutation test (перемешиваем ярлыки групп, чтобы смоделировать $H_0$ "нет разницы")

Распределение статистики выстраиваем эмпирически.

#### Расчёт p-value
$p-value$ (вероятность получить такое же или более экстремальное значение статистики) равно:

Для двусторонней альтернативы:
$$
p = \frac{\sum I(|T| \ge |T_{obs}|)}{количество \space перестановок}
$$

Для односторонней альтернативы (больше):
$$
p = \frac{\sum I(T \ge T_{obs})}{количество \space перестановок}
$$

Для односторонней альтернативы (меньше):
$$
p = \frac{\sum I(T \le T_{obs})}{количество \space перестановок}
$$

**Тренировочная задача №6**

Возьмём условие **тренировочной задачи №5**, но теперь для сравнения используем **Бутстрап и Тест перестановок**.

**Условие:** мы проверяем эффективность Push-уведомления. Целевым действием является подписка на сервис в течение трёх дней. Мы разделили сегмент на две группы, экспериментальную и контрольную, размер каждой - 1 тысяча клиентов. Экспериментальная группа получила Push-уведомление, контрольная - ничего не получала. После проведения эксперимента мы измерили количество клиентов, подписавшихся на сервис: в контрольной группе оно составило 8 клиентов, а в экспериментальной - 21 клиент.

**Задача:** сравнить конверисю в экспериментальной группе ($p_1$) с конверсией в контрольной группе ($p_2$) при помощи **Бутстрапа и Теста перестановок** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $p_1$ не равна $p_2$):
$$
H_0: p_1 = p_2 \\
H_1: p_1 <> p_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p_1$ больше $p_2$):
$$
H_0: p_1 \le p_2 \\
H_1: p_1 > p_2 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p_1$ меньше $p_2$):
$$
H_0: p_1 \ge p_2 \\
H_1: p_1 < p_2 \\
$$

In [22]:
from ab_utils.bootstrap import permutation_test_pvalue

In [23]:
n1 = 1000          # количество пользователей (испытаний) в экспериментальной группе
n2 = 1000          # количество пользователей (испытаний) в контрольной группе

x1 = 21            # количество успехов в экспериментальной группе
x2 = 8             # количество успехов в контрольной группе
alpha = 0.05       # уровень значимости alpha

p1 = float(x1) / n1     # Конверсия в экспериментальной группе
p2 = float(x2) / n2     # Конверсия в контрольной группе

x1_arr = np.zeros(n1)
x1_arr[:x1] = 1

x2_arr = np.zeros(n2)
x2_arr[:x2] = 1

print(f"Конверсия в экспериментальной группе: {p1:.1%}")
print(f"Конверсия в контрольной группе (H0): {p2:.1%}")

Конверсия в экспериментальной группе: 2.1%
Конверсия в контрольной группе (H0): 0.8%


In [26]:
alt_to_h_zero = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% больше либо равна p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% меньше либо равна p_2 {round(p2 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p_1 {round(p1 * 100.0, 1)}% НЕ равна p_2 {round(p2 * 100.0, 1)}%',
    'less': f'p_1 {round(p1 * 100.0, 1)}% меньше p_2 {round(p2 * 100.0, 1)}%',
    'greater': f'p_1 {round(p1 * 100.0, 1)}% больше p_2 {round(p2 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_manual, T_obs, T_stats = permutation_test_pvalue(
        x = x1_arr,
        y = x2_arr,
        metric_func = np.mean,
        reps = 10000,
        alternative = alt
    )

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (perm test):  {p_manual:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_manual < alpha else 'нет'}")


Альтернатива: two-sided
p-value (perm test):  0.023500; Гипотеза H_0 (p_1 2.1% равна p_2 0.8%) отвергается на уровне 0.05 в пользу H_1 (p_1 2.1% НЕ равна p_2 0.8%): да

Альтернатива: less
p-value (perm test):  0.995000; Гипотеза H_0 (p_1 2.1% больше либо равна p_2 0.8%) отвергается на уровне 0.05 в пользу H_1 (p_1 2.1% меньше p_2 0.8%): нет

Альтернатива: greater
p-value (perm test):  0.012800; Гипотеза H_0 (p_1 2.1% меньше либо равна p_2 0.8%) отвергается на уровне 0.05 в пользу H_1 (p_1 2.1% больше p_2 0.8%): да


## 2. Расчёта размера выборки (Sample Size) и минимального детектируемого эффекта (MDE)

...

## 3. Проверка корректности выбора критерия на синтетических данных

...

## 4. Построение доверительных интервалов для разницы метрик

...

## 5. Поправки при множественном тестировании

...